In [1]:
import bioread
import os
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timezone, timedelta
from pprint import pprint
from pymediainfo import MediaInfo

In [2]:
project_dir = os.getcwd().split('\\')[:-1]
project_dir = '\\'.join(project_dir)

In [3]:
project_dir

'D:\\Documenten\\Artificial Intelligence Master\\Semester 3\\tDCS_Video_Stress'

In [4]:
video_dir = project_dir + "\\data\\raw\\Video\\raw"
video_files = os.listdir(video_dir)

In [5]:
df = pd.DataFrame()

In [6]:
pps = [int(file[4:7]) for file in video_files]

In [7]:
df['pp'] = pps

In [8]:
recorded_dates = []
durations = []
for video_file in video_files:
    _dir =video_dir +'\\'+ video_file
    media_info = MediaInfo.parse(_dir)
    recorded_dates.append(media_info.tracks[0].recorded_date)
    durations.append(float(media_info.tracks[1].duration)/1000)

In [9]:
df['video_start_time'] = [datetime.strptime(date[:-6], '%Y-%m-%d %H:%M:%S').replace(tzinfo=timezone(timedelta(hours=2))) for date in recorded_dates]
df['video_duration'] = [timedelta(0,dur) for dur in durations]
df['video_end_time'] = df.video_start_time + df.video_duration

In [10]:
df['video_start_time'] = df['video_start_time'].dt.tz_convert('utc')
df['video_end_time'] = df['video_end_time'].dt.tz_convert('utc')

In [11]:
acq_files = [f'{project_dir}\\data\\raw\\Physiological\\raw\\Biopac\\pp{pp}s2.acq' for pp in pps]

In [12]:
start_time = []
duration = []
end_time = []
for i in range(len(pps)):
    acq = bioread.read_file(acq_files[i])
    acq_start_time = acq.earliest_marker_created_at
    start_time.append(acq_start_time)
    acq_total_seconds = timedelta(0,len(acq.channels[0].raw_data)/acq.channels[0].samples_per_second)
    duration.append(acq_total_seconds)
    end_time.append(acq_start_time + acq_total_seconds)

df['physio_start_time'] = start_time
df['physio_duration'] = duration
df['physio_end_time'] = end_time

In [13]:
df['physio_video_start_diff'] = df['video_start_time'].dt.tz_convert('utc') - df['physio_start_time'].dt.tz_convert('utc')

In [14]:
df

,pp,video_start_time,video_duration,video_end_time,physio_start_time,physio_duration,physio_end_time,physio_video_start_diff
0,2,2020-03-03 09:00:55+00:00,0 days 00:17:55.280000,2020-03-03 09:18:50.280000+00:00,2020-03-03 08:07:01.303000+00:00,0 days 01:29:12.305500,2020-03-03 09:36:13.608500+00:00,0 days 00:53:53.697000
1,3,2020-03-10 13:10:52+00:00,0 days 00:15:36.960000,2020-03-10 13:26:28.960000+00:00,2020-03-10 12:14:14.552000+00:00,0 days 01:32:48.979500,2020-03-10 13:47:03.531500+00:00,0 days 00:56:37.448000
2,4,2020-03-03 15:04:03+00:00,0 days 00:14:54.320000,2020-03-03 15:18:57.320000+00:00,2020-03-03 14:08:57.128000+00:00,0 days 01:29:14.838500,2020-03-03 15:38:11.966500+00:00,0 days 00:55:05.872000
3,5,2020-03-04 09:32:08+00:00,0 days 00:15:09.400000,2020-03-04 09:47:17.400000+00:00,2020-03-04 08:40:59.994000+00:00,0 days 01:25:50.768500,2020-03-04 10:06:50.762500+00:00,0 days 00:51:08.006000
4,10,2020-03-05 17:05:03+00:00,0 days 00:18:33.240000,2020-03-05 17:23:36.240000+00:00,2020-03-05 16:04:42.164000+00:00,0 days 01:40:15.584000,2020-03-05 17:44:57.748000+00:00,0 days 01:00:20.836000
5,11,2020-03-10 15:04:32+00:00,0 days 00:14:38.200000,2020-03-10 15:19:10.200000+00:00,2020-03-10 14:09:40.866000+00:00,0 days 01:35:31.103000,2020-03-10 15:45:11.969000+00:00,0 days 00:54:51.134000
6,12,2020-03-09 09:07:12+00:00,0 days 00:10:39,2020-03-09 09:17:51+00:00,2020-03-09 08:07:53.403000+00:00,0 days 01:30:48.062000,2020-03-09 09:38:41.465000+00:00,0 days 00:59:18.597000
7,13,2020-03-12 13:08:46+00:00,0 days 00:14:00.240000,2020-03-12 13:22:46.240000+00:00,2020-03-12 12:12:59.851000+00:00,0 days 01:32:51.425000,2020-03-12 13:45:51.276000+00:00,0 days 00:55:46.149000
8,14,2020-03-11 13:09:39+00:00,0 days 00:16:36.760000,2020-03-11 13:26:15.760000+00:00,2020-03-11 12:13:11.310000+00:00,0 days 01:34:39.119500,2020-03-11 13:47:50.429500+00:00,0 days 00:56:27.690000
9,17,2020-03-12 11:12:29+00:00,0 days 00:15:53.080000,2020-03-12 11:28:22.080000+00:00,2020-03-12 10:18:21.411000+00:00,0 days 01:28:56.884500,2020-03-12 11:47:18.295500+00:00,0 days 00:54:07.589000


In [15]:
timing_files = [f'{project_dir}\\data\\raw\\Timing stress session\\timing2_pp{pp}.xlsx' for pp in pps]

In [16]:
col_names = list(pd.read_excel(timing_files[1]).task +'_' + pd.read_excel(timing_files[1]).timepoint)

In [17]:
timing_values = []
for i in range(len(timing_files)):
    timing_values.append(pd.read_excel(timing_files[i]).time.values)

In [18]:
new_df = pd.DataFrame(timing_values)
new_df.columns = col_names
new_df = new_df*60

In [20]:
df = pd.concat([df, new_df], axis=1)